In [ ]:
import json
import numpy as np
from IPython.display import Audio
from matplotlib import pyplot as plt
from patch import LiveSet,SlicerPatch

Read in the default patch file, which just has the basic slicer effect.

In [ ]:
with open("default_patch.tsl","r") as f:
    all_patches = json.load(f)

Create a synthetic A440Hz sine wave.

In [ ]:
sin_freq = 440
rate = 44100
sin_b = (2*np.pi*sin_freq)/rate
n_seconds = 15
len_t = rate * n_seconds
tmp_x = np.arange(len_t)
a440_sinwave = 0.05*np.sin(sin_b*tmp_x)-0.025

Play the sine wave.

In [ ]:
Audio(a440_sinwave,rate=rate)

This function simulates the slicer effect by applying the paramters in the`STEP_LENGTH` and `STEP_LEVEL` arrays to a sound file.

In [ ]:
def simulate_slicer(pattern,sound=a440_sinwave,bpm=100,metronome=False):
    n_slices = pattern.step_number_c1
    # Step Length
    step_len = pattern.step_length_c1[:n_slices]/100
    # Step Level
    step_level = pattern.step_level_c1[:n_slices]/100
    # Seconds per beat = 1/bps = 1/(bpm/60) = 60/bpm
    spb = 60/bpm
    # Seconds per measure
    spm = 4*spb
    # samples per measure = seconds/measure * samples/second = samples/measure
    smpls_per_measure = spm * rate
    n_measures = int(np.ceil(a440_sinwave.shape[0] / smpls_per_measure))
    # Divide measure into n_slices parts
    smpls_per_slice = int(smpls_per_measure / n_slices)
    # Threshold values are the indices within each slice where we need to cut off the sound
    thresh = np.repeat(np.floor(step_len * smpls_per_slice),smpls_per_slice)
    # index values are the current index values within each slice (0 to smpls_per_slice)
    # Any time idx exceeds thresh, we cut off the sound.
    idx = np.tile(np.arange(smpls_per_slice),n_slices)
    # Mask array will be used to mask off the sound where required
    mask = (idx <= thresh).astype(np.float32)
    # Repeat the patterns for the duration of the array
    step_len_repeat = np.tile(mask,n_measures)[:a440_sinwave.shape[0]]
    step_level_repeat = np.tile(np.repeat(step_level,smpls_per_slice),n_measures)[:a440_sinwave.shape[0]]
    # Output sound
    simulated_sound = sound * step_level_repeat * step_len_repeat
    return sound * step_level_repeat * step_len_repeat

Read in the default pattern file, and parse it using the `LiveSet` object.

In [ ]:
with open("default_patch.tsl","r") as f:
    json_live_set = json.load(f)
live_set = LiveSet.from_json(json_live_set)
live_set

In [ ]:
patch = live_set.data[0][0]
patch

Generate a (very bad) simulation of the pattern.

In [ ]:
sim_sound = simulate_slicer(pattern=patch,bpm=50)

Play the sound.

In [ ]:
Audio(sim_sound,rate=rate)